In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 9, 2), datetime.date(2023, 9, 1))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-09-02
yesterday: 2023-09-01 00:00:00


In [3]:
yesterday = yesterday.date()
a_year_ago = yesterday - timedelta(days=365)
yesterday, a_year_ago

(datetime.date(2023, 9, 1), datetime.date(2022, 9, 1))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [4]:
pd.read_sql_query('SELECT COUNT(*) AS records FROM sales', conlite)

,records
0,5178


In [5]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

5931

In [6]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'BEM', 'BH', 'CK', 'CPN', 'CPNREIT', 'DELTA', 'DIF', 'GVREIT', 'HANA', 'ICHI', 'III', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTB', 'M', 'MCS', 'NER', 'ORI', 'PSH', 'PTG', 'PTTEP', 'PTTGC', 'QH', 'RCL', 'SAPPE', 'SC', 'SCC', 'SENA', 'SINGER', 'SIRI', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'TTB', 'WHAIR', 'WHART'"

In [7]:
len(in_p.split(','))

48

### Get past one year data

In [8]:
sql = """
SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (a_year_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '2022-09-01' AND name IN ('AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'BEM', 'BH', 'CK', 'CPN', 'CPNREIT', 'DELTA', 'DIF', 'GVREIT', 'HANA', 'ICHI', 'III', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTB', 'M', 'MCS', 'NER', 'ORI', 'PSH', 'PTG', 'PTTEP', 'PTTGC', 'QH', 'RCL', 'SAPPE', 'SC', 'SCC', 'SENA', 'SINGER', 'SIRI', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'TTB', 'WHAIR', 'WHART') 
ORDER BY name, date


In [9]:
df = pd.read_sql(sql, const)
df.head()

,name,date,price,qty,maxp,minp
0,AH,2022-09-01,25.00,2968232,25.25,24.60
1,AH,2022-09-02,27.25,14510767,27.50,24.80
2,AH,2022-09-05,26.75,9912932,27.75,26.50
3,AH,2022-09-06,27.75,4878824,27.75,26.25
4,AH,2022-09-07,26.50,9254853,28.50,26.00


In [10]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

### Create monitors from orders

In [11]:
sql = """
SELECT name,trade 
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
orders = pd.read_sql(sql, conlite)
orders.set_index(['name'],inplace=True)
orders

,trade
name,
AH,S
AIMIRT,S
ASK,S
ASP,S
AWC,S
BBL,B
BCH,S
BEM,B
BH,B


In [12]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders.to_csv(data_file, header=None)
orders.to_csv(output_file)
orders.to_csv(box_file)

In [13]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders
GROUP BY trade
ORDER BY trade
"""
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,32
1,S,16


### For new orders that never have XXX.html & XXX.CSV

In [14]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [15]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(152, 1)

In [16]:
df_merge = pd.merge(orders, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(162, 3)

In [17]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge
0,AH,S,left_only
4,AWC,S,left_only
7,BEM,B,left_only
10,CPN,B,left_only
24,M,B,left_only
28,PSH,B,left_only
32,QH,B,left_only
34,SAPPE,B,left_only
39,SIRI,B,left_only
44,TOA,B,left_only


In [18]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(114, 3)

In [19]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks.shape

(48, 2)

In [20]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [21]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BBL,O
BEM,O
BH,O
CK,O
CPN,O
DELTA,O
DIF,B
HANA,O
ICHI,O


In [22]:
buy_candidates.shape[0]

32

In [23]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
AH,I
AIMIRT,I
ASK,I
ASP,I
AWC,I
BCH,I
CPNREIT,S
GVREIT,I
IVL,I


In [24]:
sell_candidates.shape[0]

16